# Pull in the transliteration table (Ethiopic --> Latin)

In [ ]:
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get('https://scriptsource.org/cms/scripts/page.php?item_id=entry_detail&uid=vsytndbyev', headers=headers)

In [ ]:
import pandas as pd

results = pd.read_html(response.text)
df = results[1]
df.head()

In [ ]:
df.Glyph = df.USV.apply(lambda val: chr(int(val, 16)))
df.head()

In [ ]:
df.to_csv("transliteration-table.csv", index=False)

## Simplify Transliteration Table

In [ ]:
df = pd.read_csv("transliteration-table.csv")
df = df.drop(columns=['USV'])
df = df.set_index('Glyph')
df

In [ ]:
!pip install Unidecode

In [ ]:
import unidecode
import numpy as np

def simplify(value):
    try:
        return unidecode.unidecode(value).lower()
    except:
        return np.nan
    
simplified = df.applymap(simplify)
merged = df.join(simplified, rsuffix='_simp')
merged.head(10)

In [ ]:
uniqued = df.fillna('').apply(lambda row: list(set([v for v in row.values if '/' not in str(v)])), axis=1)
uniqued

In [ ]:
uniqued.to_json("lookup-table.json")

In [ ]:
import json

with open("lookup-table.json") as f:
    lookup = json.load(f)

punctuation = [
    ['።', '.'],
    ['፡', ' '],
    ['፣', ','],
    ['፤', ';'],
    ['፥', ':'],
    ['፧', '?']
]

for punc in punctuation:
    lookup[punc[0]] = [punc[1]]

lookup

In [ ]:
lookup_simp = {}
for key in lookup.keys():
    values = list(set([unidecode.unidecode(v).lower() for v in lookup[key]]))
    lookup_simp[key] = [v for v in values if '@' not in v]
    
lookup_simp

# Create reverse dictionary (Latin --> Ethiopic)

In [ ]:
# create reverse dictionary
lat_eth = dict()

for key in lookup_simp.keys():
    values = lookup_simp[key]
    for v in values:
        if v in lat_eth.keys():
            new_list = lat_eth[v].copy()
            new_list.append(key)
            lat_eth[v] = new_list
        else:
            lat_eth[v] = [key]

lat_eth

# Transliteration work

In [ ]:
# load dictionary to prune transliteration options 
am_dic_file = open("am_dic.txt", "r", encoding="utf-8")
am_dic = []

for w in am_dic_file.readlines():
    am_dic.append(w.rstrip())

am_dic_file.close()
am_dic = set(am_dic)

## Use CountVectorizer() to do char n grams

In [ ]:
# clean corpus
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

file = open("raw/new-am.txt", "r", encoding="utf-8")
corpus = file.read()

# tokenize corpus (https://machinelearningmastery.com/clean-text-machine-learning-python/)
tokens = word_tokenize(corpus)
# remove all tokens that are not alphabetic
tokens = [word for word in tokens if word.isalpha()]

In [ ]:
# train model to do ngram work
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 3), analyzer="char_wb")
cv_fit = cv.fit_transform(tokens)

In [ ]:
print("Vocabulary size 1-3gram: ", len(cv.vocabulary_))

In [ ]:
import numpy as np

ngram_list = cv.get_feature_names()
count_list = np.asarray(cv_fit.sum(axis=0))[0]

# make a dictionary with frequencies 
freq_dict = dict(zip(ngram_list,count_list))

# get unigram, bigram, trigram total counts
unigram_count = 0
bigram_count = 0
trigram_count = 0

for key in freq_dict.keys():
    if len(key)==1:
        unigram_count += freq_dict[key]
    elif len(key)==2:
        bigram_count += freq_dict[key]
    else:
        trigram_count += freq_dict[key]

print("Unigram Count: ", unigram_count)
print("Bigram Count: ", bigram_count)
print("Trigram Count: ", trigram_count)

## Actual Transliterating

In [ ]:
import json

with open('lat_eth.json', 'w') as f:
    json.dump(lat_eth, f)

In [ ]:
def get_breakdowns(segment):
    options = []
    max_len = min([len(segment), 4])
    for i in range(1, max_len+1):
        potential = segment[:i]
        if potential in lat_eth.keys():
            remainder = segment[i:]
            if remainder == "":
                options.append([potential])
            else:
                enders = get_breakdowns(remainder)
                if enders == []:
                    return []
                else:
                    options.extend([potential, *e] for e in enders)
    return options
    

breakdowns = get_breakdowns("inidihi")
breakdowns

In [ ]:
import itertools

def convert(segmentation):
    final_list = []
    relevant_lists = []
    for elmt in segmentation:
        relevant_lists.append(lat_eth[elmt])
    for i in itertools.product(*relevant_lists):
        final_list.append(''.join(i))
    return final_list

In [ ]:
eth_options = convert(['i', 'n', 'i', 'd', 'i', 'h', 'i'])
eth_options[0:5]

In [ ]:
import json

with open("char2idx.json") as f:
    char2idx = json.load(f)

with open("idx2char.json") as f:
    idx2char = json.load(f)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import keras

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model = tf.keras.models.load_model("char_model", compile=False)

model.compile(optimizer='adam', loss=loss)


In [ ]:
tf.__version__

In [ ]:
tf.keras.__version__

In [ ]:
model.build(tf.TensorShape([1, None]))

In [ ]:
model

In [ ]:
start_string = "በ"
temperature = 1.0
num_generate = 1
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

model.reset_states()
predictions = model(input_eval)
predictions = tf.squeeze(predictions, 0)
predictions = predictions / temperature

predicted_id = np.argsort(predictions)[0, -3:]
predicted_id
print("".join(idx2char[predicted_id]))

In [55]:
# generate possible transliterations

import string
import itertools
import operator
import string 
import random

# given a sentence in latin characters, splits and sends word by word to the 
# function transliterate_word
def transliterate(text):
    sent_trans = []

    sentence = [i for j in text.split() for i in (j, ' ')][:-1]
    cleaned = []
    for elmt in sentence:
      elmt_tokenized = word_tokenize(elmt)
      if elmt == ' ':
        cleaned.append(' ')
      elif len(elmt) == len(elmt_tokenized):
        cleaned.append(elmt)
      else:
        for i in elmt_tokenized:
          if i == "'":
            elmt_tokenized.remove(i)
        cleaned += elmt_tokenized

    for word in cleaned:
      sent_trans.append(transliterate_word(word))
    
    return "".join(sent_trans)


# transliterate_word returns spaces/punctuations as appropriate
# and sends an actual latin character word to ngram_selected(word) to 
# obtain the appropriate transliterated word in ethiopic
def transliterate_word(word):
    # print("transliterating the word")
    if word in string.punctuation and word not in lat_eth.keys():
      return word
    elif word.isnumeric() == True:
      return word
    elif word == " ":
      return word
    elif len(word) > 15:
      return word
    else:
      word = unidecode.unidecode(word).lower()
      # print(f"Sending {word} to ngram_selected")
      return ngram_selected(word)

# ngram_selected takes a latin character word and generates all possible ethiopic
# transliterations by calling the function possibilities; it then selects the 
# ethiopic option with the highest score using the function word_score
def ngram_selected(word):
    options = possibilities(word)
    # print('possibilities are', options)
    if len(options) == 0:
      return word
    else:
      scores = dict()
      for opt in options:
        score = word_score(opt)
        scores[opt] = score
      # print('scores were calculated as', scores)
      selected_word = max(scores.items(), key=operator.itemgetter(1))[0]
      return selected_word

# the function possibilities takes a latin character word and returns all 
# possible transliterations into ethiopic based on the reverse dictionary
# this function calls the function prune to remove entries that are 
# not in an actual amharic dictionary (unless pruning results in 0 options)   
# this function also calls the function convert to go from latin char to 
# ethiopic char as based on the reverse dictionary
def possibilities(word):
#     print("calculating possibilities")
    # split word into chars
    chars = list(word)
    
    # generate all combinations 
    # https://stackoverflow.com/questions/27263155/python-find-all-possible-
    # word-combinations-with-a-sequence-of-characters-word
#     print("all combinations of", chars)
    combinatorics = itertools.product([True, False], repeat=len(chars) - 1)
    latin_segmentation = []
    add = True
    for combination in combinatorics:
#         print("looking at", combination)
        i = 0
        one_such_combination = [chars[i]]
        for slab in combination:
            # print("slab is", slab)
            i += 1
            if not slab: # there is a join
                one_such_combination[-1] += chars[i]
            else:
                one_such_combination += [chars[i]]
        
        for elmt in one_such_combination:
#             print("Looking up", elmt)
            if elmt not in lat_eth.keys():
                add = False
                break
        # only add/consider if segmentation can be converted into ethiopic 
        # characters
        if add == True:  
            latin_segmentation.append(one_such_combination)
            
        # reset
        add = True
    
    # conversion
    ethiopic_opts = []
    for segmentation in latin_segmentation:
        ethiopic_opts += convert(segmentation)

    pruned = prune(ethiopic_opts)
    if len(pruned) == 0:
      if len(ethiopic_opts) < 100:
        return ethiopic_opts
      else:
        sampling = random.choices(ethiopic_opts, k=99)
        return sampling
    else:
      return pruned

# this is called by the function possibilities to convert from latin char
# to ethiopic char given a particular segmentation (i.e. i-di vs. i-d-i might
# both be sent separately)
def convert(segmentation):
    final_list = []
    relevant_lists = []
    for elmt in segmentation:
        relevant_lists.append(lat_eth[elmt])
    for i in itertools.product(*relevant_lists):
        final_list.append(''.join(i))
    return final_list

# this is called by the function possibilities to prune the list of possible
# ethiopic transliterations
def prune(possibilities):
    final_possibilities = []
    for candidate in possibilities: 
      if candidate in am_dic:
          final_possibilities.append(candidate)
    return final_possibilities

# this function is called by ngram_selected to determine the probability of
# an ethiopic word occurring (using ngram counts)
# this function calls get_ngrams to split the given word into n-length 
# subsections for scoring
# this function also calls one or multiple of the [n]gram_probability functions 
# to compute each [n]gram score, which are then weighted evenly in computing 
# the final score
def word_score(word):
  sequence = list(word)
  if len(word) >= 3:
    # calculate trigram probability
    w = 1/3.0
    trigrams = get_ngrams(sequence, 3)
    bigrams = get_ngrams(sequence, 2)
    unigrams = get_ngrams(sequence, 1)
    score_t = trigram_probability(trigrams)
    score_b = bigram_probability(bigrams)
    score_u = unigram_probability(unigrams)
    score = (w*score_t)+(w*score_b)+(w*score_u) 
  elif len(word) >= 2:
    # calculate bigram probability
    w = 1/2.0
    bigrams = get_ngrams(sequence, 2)
    unigrams = get_ngrams(sequence, 1)
    score_b = bigram_probability(bigrams)
    score_u = unigram_probability(unigrams)
    score = (w*score_b)+(w*score_u) 
  else:
    # calculate unigram probability
    unigrams = get_ngrams(sequence, 1)
    score = unigram_probability(word)
  return score

# called by the function word_score to generate n gram subsections 
# from a given ethiopic word
def get_ngrams(sequence, n):
    input = sequence
    output = []
    for i in range(0, len(input) - n + 1):
        output.append(input[i:i + n])
    
    return [''.join(l) for l in output]

# these funtions are called by word_score to compute [n]gram probabilities given
# an ethiopic word 
def trigram_probability(trigrams):
  freq = 0
  for t in trigrams:
    # get freq
    if t in freq_dict:
      freq += freq_dict[t]
  avg_prob = freq/(len(trigrams) * trigram_count)
  return avg_prob

def bigram_probability(bigrams):
  freq = 0
  for b in bigrams:
    # get freq
    if b in freq_dict:
      freq += freq_dict[b]
  avg_prob = freq/(len(bigrams) * bigram_count)
  return avg_prob

def unigram_probability(unigrams):
  freq = 0
  for u in unigrams:
    # get freq
    if u in freq_dict:
      freq += freq_dict[u]
  avg_prob = freq/(len(unigrams) * unigram_count)
  return avg_prob

In [56]:
ngram_selected("inidihi")

'እንዲህ'

In [57]:
transliterate("inidihi sil . hulat sewochi liseliyu wade")
#እንዲህ ሲል ። ሁለት ሰዎች ሊጸልዩ ወደ

'እንዲህ ስል ። ሁለት ሰዎች ሊጸልዩ ወደ'

In [58]:
transliterate("ijasusme . 'eweneti iwnat `elacehualehu")
#ኢየሱስም ። እውነት እውነት እላችኋለሁ

'ኢየሱስም ። እውንት እውነት እላችኋለሁ'

In [59]:
transliterate("abatune wejeme ina'tun ajakebirm tlalacihu ;")
#አባቱን ወይም እናቱን አያከብርም ትላላችሁ ፤ 

'አባቱን ወይም እናቱን አያከብእርም ትላላችሁ ፤'

In [60]:
transliterate("bamine 'ajenetim mote ymoti zendi `enidalawe")
#በምን ዓይነትም ሞት ይሞት ዘንድ እንዳለው

'በምን አይነትም ሞት ይሞት ዘንድ እንዳለው'

# TEST

In [ ]:
! pip install python-levenshtein

In [61]:
# EVALUATION

import Levenshtein
from statistics import mean

# this function takes two files (parallel texts) and generates
# two lists after stripping beginning/trailing whitespace
def extract_from_files(ethiopic_file, latin_file):
  eth_file = open(ethiopic_file)
  lat_file = open(latin_file)
  ethiopic = [line.rstrip() for line in eth_file.readlines()]
  latin = [line.rstrip() for line in lat_file.readlines()] 
  
  ethiopic = list(ethiopic)
  latin = list(latin)

  return ethiopic, latin

# this function takes two parallel lists and evaluates how
# our model performs
def evaluate(ethiopic, latin):
  accuracies = []

  count = 0
  for line in latin:
    predict = transliterate(line)
    correct = ethiopic[count]
    accuracy = Levenshtein.ratio(predict,correct)
    accuracies.append(accuracy) 
    count += 1

  return mean(accuracies)

In [96]:
import re
def evaluate_top_n(ethiopic, latin, top_n=2):
    accuracies = []

    total = 0
    correct = 0
    for eth_line, lat_line in zip(ethiopic, latin):
        eth_tokens = re.split('\s+', eth_line.strip())
        lat_tokens = re.split('\s+', lat_line.strip())
        for eth_token, lat_token in zip(eth_tokens, lat_tokens):
            results = transliterate_word(lat_token, top_n=3)
            print(eth_token, results, lat_token)
            if eth_token in results:
                correct += 1
            total += 1
    return correct/total

def transliterate_word(word, top_n=1):
    if word.isspace():
        return word
    if word in lat_eth.keys() and len(lat_eth[word]) == 0:
        return lat_eth[word][0]
    elif word in string.punctuation:
        return word
    elif word.isnumeric() == True:
        return word
    else:
        result = ngram_selected(word)
        return [result]

## first on google translated pairs

In [97]:
ethiopic_tot, latin_tot = extract_from_files("raw/original.txt", "raw/transliterated.txt")

In [98]:
ethiopic_2 = ethiopic_tot[0:1]
latin_2 = latin_tot[0:1]

In [99]:
evaluate_top_n(ethiopic_2, latin_2)

“የዳታ ['የዳታ'] jadata
ፕሮሰሲንግ ['ጵርውኦጸስንኤግእ'] perwosasinegi
አገልግሎት” ['አገልግሎት'] `ageligelwote
ማለት ['ማለት'] maleti
በኮምፒዩተር ['በቾምኤፕእይዑጥአርእ'] becomepiyutari
ሥርዓት ['ስርአት'] siri`at
አማካኝነት ['አማካኝነት'] `amakannate
ዳታን ['ዳታን'] datan
የመቀበል ['የምችዓበል'] yemecabal
፣ , ,
የማከማቸት ['ያምዓችማከት'] jamacemaket
፣ , ,
የመተንተን ['ጅአመትአንትኧን'] jamatanitan
፣ , ,
የማሰራጨት ['የመሰረት'] yemasarat
፣ , ,
የማጓጓዝ ['የምአግወግውአዝ'] yamagwagwaz
ወይም ['ወይም'] wayim
የማስተላለፍ ['የማስተላለፍ'] yemastelalafi
አገልግሎት ['አገልግሎት'] 'agelglwoti
ሲሆን ['ስሆን'] sihwone
የኔትዎርክ ['የንተዖርኢክ'] yeneteworik
አገልግሎችንም ['አጘልገልኦንመ'] agaligelonme


0.2608695652173913

In [67]:
ethiopic_3 = ethiopic_tot[0:100]
latin_3 = latin_tot[0:100]

In [68]:
%%time
evaluate(ethiopic_3, latin_3)

CPU times: user 4min 47s, sys: 32.5 s, total: 5min 19s
Wall time: 5min 52s


0.7330918000222264

In [69]:
ethiopic = ethiopic_tot[0:1000]
latin = latin_tot[0:1000]

In [70]:
%%time
evaluate(ethiopic,latin)

KeyboardInterrupt: 

In [ ]:
num_words = 0

for line in ethiopic_tot[0:1000]:
  num_words += len(line)
num_words

## now on manually generated parallel texts

In [ ]:
ethiopic_manual, latin_manual = extract_from_files('taitu_am.txt','taitu_rom.txt')

In [ ]:
evaluate(ethiopic_manual, latin_manual)